### Coleta usando a API do twitter

In [20]:
#Coleta
import oauth2
print(oauth2.Consumer)

<class 'oauth2.Consumer'>


In [21]:
import json
import pandas as pd
import csv
import nltk
from nltk.tokenize import TweetTokenizer

In [4]:
def criaJsonSaida(json_input):
    return {
        "id": json_input["id"],
        "data_criacao": json_input["created_at"],
        "text": json_input["full_text"].lower(),
        "usuario": json_input["user"]["name"],
        "e_sobre_a_empresa": None
    }

def get_requisicao(cliente, max_id, termo):
    requisicao = cliente.request(
        'https://api.twitter.com/1.1/search/tweets.json?q={termo}&tweet_mode=extended&result_type=recent&lang=pt-br&count=3&{max_id}'.format(max_id=max_id, termo=termo))
    decodificar = requisicao[1].decode()
    try:
        objeto = json.loads(decodificar)
        return objeto['statuses']
    except Exception:
        print(objeto)

def coleta(api_key, api_secret_key, token_key, token_secret_key, termo, num_tweets, arquivo_saida):
    consumer = oauth2.Consumer(api_key, api_secret_key)
    token = oauth2.Token(token_key, token_secret_key)
    cliente = oauth2.Client(consumer, token)
    strUrl = ""
    max_id = ""
    num_tweets_collected = 0

    tweetsColetados = {
        "tweets": []
    }

    mantem_tweeets_nao_repetidos = set()
    
    try:
        # coleta os tweets
        while(num_tweets_collected < num_tweets):
            tweets = get_requisicao(cliente, max_id, termo)            
            for tweet in tweets:
                tweet_saida = criaJsonSaida(tweet)
                par_unico = (tweet_saida["usuario"], tweet_saida["data_criacao"])
                if par_unico not in mantem_tweeets_nao_repetidos:
                    mantem_tweeets_nao_repetidos.add(par_unico)
                    tweetsColetados["tweets"] += [criaJsonSaida(tweet)]
                
                max_id = "max_id={max_id}&".format(max_id=tweet['id'])
                num_tweets_collected += 1
    except Exception as e:
        print(e)
    finally:
        with open(arquivo_saida, 'w') as f:
            f.write(json.dumps(tweetsColetados["tweets"], indent=2))


In [5]:
# inicializa keys
api_key = 'suaAPIKey'
api_secret_key = 'suaAPISecretKey'
token_key = 'suaTokenKey'
token_secret_key = 'suaTokenSecretKey'
termo = 'sadia'
num_tweets = 100
arquivo_com_tweetes_coletados = 'tweets_coletados.json'

try:
    coleta(api_key, api_secret_key, token_key,
           token_secret_key, termo, num_tweets, arquivo_com_tweetes_coletados)
except Exception as e:
    print(e)

### Gerando csv com os tweets a serem rotulados

In [8]:
with open(arquivo_com_tweetes_coletados, 'r') as f:
    dic_tweets = json.loads(f.read())
    tweetsDF = pd.read_json(json.dumps(dic_tweets))
nome_arquivo_com_tweetes_rotulados = 'tweets_coletados.csv'
tweetsDF.to_csv(nome_arquivo_com_tweetes_rotulados, quoting=csv.QUOTE_ALL, index=False)

### Lendo o csv com os tweets rotulados

In [9]:
tweets_rotuladosDF = pd.read_csv(nome_arquivo_com_tweetes_rotulados,quoting=csv.QUOTE_ALL)

Quantidade de tweets por classe

In [11]:
sobre_a_sadia_qtd = tweets_rotuladosDF[tweets_rotuladosDF.e_sobre_a_empresa == 1]['e_sobre_a_empresa'].count()
sobre_outros_qtd = tweets_rotuladosDF['e_sobre_a_empresa'].count() - sobre_a_sadia_qtd
print("Sobre a Sadia: " +str(sobre_a_sadia_qtd) + " tweets")
print("Sobre outro assunto: " +str(sobre_outros_qtd) + " tweets")

Sobre a Sadia: 0 tweets
Sobre outro assunto: 0 tweets


### Identificando os brigramas e trigramas encontrados nos twittes (TODO)

In [39]:
n = 0
for indice, linha in tweets_rotuladosDF.iterrows() :
    if( n == 5):
        break
    print(indice)
    print(linha)
    print("---------")
    n += 1

0
data_criacao                            Sun Apr 21 21:43:51 +0000 2019
e_sobre_a_empresa                                                    1
id                                               1,12008073153398E+018
text                 rt @calazani: com quantos anos você descobriu ...
usuario                                               Daniela Pinheiro
Name: 0, dtype: object
---------
1
data_criacao                            Sun Apr 21 21:41:28 +0000 2019
e_sobre_a_empresa                                                    1
id                                                1,1200801331871E+018
text                 rt @calazani: com quantos anos você descobriu ...
usuario                                                   Filipe Souza
Name: 1, dtype: object
---------
2
data_criacao                            Sun Apr 21 21:37:04 +0000 2019
e_sobre_a_empresa                                                    1
id                                               1,12007902504773E+018
text 